# Match regional geodetic mass balance
The OGGM function `match_regional_geodetic_mb()` shifts the mass balance residuals $\beta^*$ to match the observations presented by [Davaze et al. 2020](https://www.frontiersin.org/articles/10.3389/feart.2020.00149/full). Hereafter I'll have a look at it and re-implement it to work with the VAS model

## OGGM flowline model setup

In [ ]:
# import internal and externals libraries
import os
import numpy as np
import pandas as pd
import xarray as xr

import logging

log = logging.getLogger('equilibrium-runs')

# import the needed OGGM modules
from oggm import cfg, utils, workflow
from oggm.core import gis, climate, flowline


log.info('Starting run')

# specify glaciers by RGI IDs (INPUT)
rgi_ids = ['RGI60-11.00897']

# compute RGI region and version from RGI IDs
# assuming all RGI IDs are from within one version and region
rgi_region = (rgi_ids[0].split('-')[-1]).split('.')[0]
rgi_version = (rgi_ids[0].split('-')[0])[-2:-1]

# load default parameter file
cfg.initialize()

# specify path to working and output directories (INPUT)
WORKING_DIR = '/Users/oberrauch/work/master/working_directories/oggm_run/'
OUTPUT_DIR = '/Users/oberrauch/work/master/data/oggm_run/'

# create working directory
utils.mkdir(WORKING_DIR)
utils.mkdir(OUTPUT_DIR)
# set path to working directory
cfg.PATHS['working_dir'] = WORKING_DIR
# set RGI version and region
cfg.PARAMS['rgi_version'] = rgi_version
# define how many grid points to use around the glacier,
# if you expect the glacier to grow large use a larger border
cfg.PARAMS['border'] = 10
# use default climate (=CRU)
# for HISTALP uncomment the following lines (INPUT)
# cfg.PARAMS['baseline_climate'] = 'HISTALP'
# cfg.PARAMS['prcp_scaling_factor'] = 1.75
# cfg.PARAMS['temp_melt'] = -1.75
# cfg.PARAMS['run_mb_calibration'] = False

# the bias is defined to be zero during the calibration process,
# which is why we don't use it here to reproduce the results
cfg.PARAMS['use_bias_for_run'] = True
# set minimum ice thickness to include in glacier length computation
# this reduces weird spikes in length records
cfg.PARAMS['min_ice_thick_for_length'] = 0.1

# read RGI entry for the glaciers as DataFrame
# containing the outline area as shapefile
rgidf = utils.get_rgi_glacier_entities(rgi_ids)

# get and set path to intersect shapefile
intersects_db = utils.get_rgi_intersects_region_file(region=rgi_region)
cfg.set_intersects_db(intersects_db)

# sort by area for more efficient parallel computing
rgidf = rgidf.sort_values('Area', ascending=False)
cfg.PARAMS['use_multiprocessing'] = True
# operational run, all glaciers should run
cfg.PARAMS['continue_on_error'] = True

# initialize the GlacierDirectory
gdirs = workflow.init_glacier_directories(rgidf, reset=False, force=True)

# run gis tasks
workflow.gis_prepro_tasks(gdirs)
# run climate tasks
workflow.execute_entity_task(climate.process_climate_data, gdirs)
# compute local t* and the corresponding mu*
workflow.execute_entity_task(climate.local_t_star, gdirs)


workflow.execute_entity_task(climate.mu_star_calibration, gdirs)
# run inversion tasks
workflow.inversion_tasks(gdirs)
# finalize preprocessing
workflow.execute_entity_task(flowline.init_present_time_glacier, gdirs)

Run `match_regional_geodetic_mb()` just to see if it works and what its output is...

In [ ]:
from oggm import workflow
workflow.match_regional_geodetic_mb(gdirs, rgi_region)

## VAS model setup

In [1]:
# import internal and externals libraries
import os
import numpy as np
import pandas as pd
import xarray as xr

import logging

log = logging.getLogger('vas-template')

# import the needed OGGM modules
from oggm import cfg, utils, workflow
from oggm.core import gis, climate, flowline
import oggm_vas as vascaling

log.info('Starting run')

# specify glaciers by RGI IDs (INPUT)
rgi_ids = ['RGI60-11.00897']

# compute RGI region and version from RGI IDs
# assuming all they are all the same
rgi_region = (rgi_ids[0].split('-')[-1]).split('.')[0]
rgi_version = (rgi_ids[0].split('-')[0])[-2:-1]

# load default parameter file
vascaling.initialize()

# get environmental variables for working and output directories
WORKING_DIR = '/Users/oberrauch/work/master/working_directories/vas_run/'
OUTPUT_DIR = '/Users/oberrauch/work/master/data/vas_run/'

# create working directory
utils.mkdir(WORKING_DIR)
utils.mkdir(OUTPUT_DIR)
# set path to working directory
cfg.PATHS['working_dir'] = WORKING_DIR
# set RGI version and region
cfg.PARAMS['rgi_version'] = rgi_version
# define how many grid points to use around the glacier,
# if you expect the glacier to grow large use a larger border
cfg.PARAMS['border'] = 20
# we use HistAlp climate data
cfg.PARAMS['baseline_climate'] = 'HISTALP'
# set the mb hyper parameters accordingly
cfg.PARAMS['prcp_scaling_factor'] = 2.5
cfg.PARAMS['temp_melt'] = -0.5
cfg.PARAMS['run_mb_calibration'] = False
# set minimum ice thickness to include in glacier length computation
# this reduces weird spikes in length records
cfg.PARAMS['min_ice_thick_for_length'] = 0.1

# the bias is defined to be zero during the calibration process,
# which is why we don't use it here to reproduce the results
cfg.PARAMS['use_bias_for_run'] = True

# read RGI entry for the glaciers as DataFrame
# containing the outline area as shapefile
rgidf = utils.get_rgi_glacier_entities(rgi_ids)

# get and set path to intersect shapefile
intersects_db = utils.get_rgi_intersects_region_file(region=rgi_region)
cfg.set_intersects_db(intersects_db)

# sort by area for more efficient parallel computing
rgidf = rgidf.sort_values('Area', ascending=False)
cfg.PARAMS['use_multiprocessing'] = True
# operational run, all glaciers should run
cfg.PARAMS['continue_on_error'] = False

# initialize the GlacierDirectory
gdirs = workflow.init_glacier_directories(rgidf, reset=False, force=True)

# define the local grid and glacier mask
workflow.execute_entity_task(gis.define_glacier_region, gdirs,
                             source=None)
workflow.execute_entity_task(gis.glacier_masks, gdirs)
# process the given climate file
workflow.execute_entity_task(climate.process_climate_data, gdirs)
# compute local t* and the corresponding mu*
workflow.execute_entity_task(vascaling.local_t_star, gdirs)

2021-01-21 16:43:57: oggm.cfg: Reading default parameters from the OGGM `params.cfg` configuration file.
2021-01-21 16:43:57: oggm.cfg: Multiprocessing switched ON according to the parameter file.
2021-01-21 16:43:57: oggm.cfg: Multiprocessing: using all available processors (N=4)
2021-01-21 16:43:57: oggm.utils: Checking the download verification file checksum...
2021-01-21 16:43:58: oggm.cfg: WARNING: adding an unknown parameter `vas_c_area_m2`:`0.191` to PARAMS.
2021-01-21 16:43:58: oggm.cfg: WARNING: adding an unknown parameter `vas_gamma_area`:`1.375` to PARAMS.
2021-01-21 16:43:58: oggm.cfg: WARNING: adding an unknown parameter `vas_c_length_m`:`4.5507` to PARAMS.
2021-01-21 16:43:58: oggm.cfg: WARNING: adding an unknown parameter `vas_q_length`:`2.2` to PARAMS.
2021-01-21 16:43:58: oggm.cfg: PARAMS['rgi_version'] changed from `61` to `6`.
2021-01-21 16:43:58: oggm.cfg: PARAMS['baseline_climate'] changed from `CRU` to `HISTALP`.
2021-01-21 16:43:58: oggm.cfg: PARAMS['temp_melt'] 

[None]

2021-01-21 16:44:19: oggm_vas.core: (RGI60-11.00897) fixed_geometry_mass_balance
2021-01-21 16:45:06: oggm.utils: (RGI60-11.00897) glacier_statistics


In [3]:
# Get the mass-balance VAS would give out of the box
df = vascaling.compile_fixed_geometry_mass_balance(gdirs, path=False)
df = df.dropna(axis=0, how='all').dropna(axis=1, how='all')

2021-01-21 16:44:19: oggm.workflow: Execute entity task fixed_geometry_mass_balance on 1 glaciers


In [5]:
# And also the Area and calving fluxes
dfs = utils.compile_glacier_statistics(gdirs, path=False)
odf = pd.DataFrame(df.loc[2006:2018].mean(), columns=['SMB'])
odf['AREA'] = dfs.rgi_area_km2 * 1e6

2021-01-21 16:45:06: oggm.workflow: Execute entity task glacier_statistics on 1 glaciers


In [8]:
rgi_reg = '11'

In [9]:
rdf = 'rgi62_areas.csv'
rdf = pd.read_csv(utils.get_demo_file(rdf), dtype={'O1Region': str})
ref_area = rdf.loc[rdf['O1Region'] == rgi_reg].iloc[0]['AreaNoC2NoNominal']
diff = (1 - odf['AREA'].sum() * 1e-6 / ref_area) * 100

,O1Region,Area,IsNominal,IsLandTerminating,IsTidewater,IsLakeTerminating,IsShelfTerminating,IsMarineTerminating,AreaNoC2,AreaNoC2NoNominal
10,11,2092.146,2,3927,0,0,0,0,2092.146,2091.98


In [2]:
# Get the mass-balance OGGM would give out of the box
df = utils.compile_fixed_geometry_mass_balance(gdirs, path=False)
df = df.dropna(axis=0, how='all').dropna(axis=1, how='all')

2021-01-21 16:02:06: oggm.workflow: Execute entity task fixed_geometry_mass_balance on 1 glaciers


FileNotFoundError: [Errno 2] No such file or directory: '/Users/oberrauch/work/master/working_directories/vas_run/per_glacier/RGI60-11/RGI60-11.00/RGI60-11.00897/inversion_flowlines.pkl'

In [ ]:
def fixed_geometry_mass_balance(gdir, ys=None, ye=None, years=None,
                                monthly_step=False,
                                use_inversion_flowlines=True,
                                climate_filename='climate_historical',
                                climate_input_filesuffix=''):
    """Computes the mass-balance with climate input from e.g. CRU or a GCM.

    Parameters
    ----------
    gdir : :py:class:`oggm.GlacierDirectory`
        the glacier directory to process
    ys : int
        start year of the model run (default: from the climate file)
        date)
    ye : int
        end year of the model run (default: from the climate file)
    years : array of ints
        override ys and ye with the years of your choice
    monthly_step : bool
        whether to store the diagnostic data at a monthly time step or not
        (default is yearly)
    use_inversion_flowlines : bool
        whether to use the inversion flowlines or the model flowlines
    climate_filename : str
        name of the climate file, e.g. 'climate_historical' (default) or
        'gcm_data'
    climate_input_filesuffix: str
        filesuffix for the input climate file
    """

    if monthly_step:
        raise NotImplementedError('monthly_step not implemented yet')

    mb = MultipleFlowlineMassBalance(gdir, mb_model_class=PastMassBalance,
                                     filename=climate_filename,
                                     use_inversion_flowlines=use_inversion_flowlines,
                                     input_filesuffix=climate_input_filesuffix)

    if years is None:
        if ys is None:
            ys = mb.flowline_mb_models[0].ys
        if ye is None:
            ye = mb.flowline_mb_models[0].ye
        years = np.arange(ys, ye + 1)

    odf = pd.Series(data=mb.get_specific_mb(year=years),
                    index=years)
    return odf

In [ ]:
def match_regional_geodetic_mb(gdirs, rgi_reg):
    """Regional shift of the mass-balance residual to match observations.

    This is useful for operational runs, but also quite hacky.
    Let's hope we won't need this for too long.

    Parameters
    ----------
    gdirs : the list of gdirs (ideally the entire region_
    rgi_reg : str
       the rgi region to match
    """

    # Get the mass-balance OGGM would give out of the box
    df = utils.compile_fixed_geometry_mass_balance(gdirs, path=False)
    df = df.dropna(axis=0, how='all').dropna(axis=1, how='all')

    # And also the Area and calving fluxes
    dfs = utils.compile_glacier_statistics(gdirs, path=False)
    odf = pd.DataFrame(df.loc[2006:2018].mean(), columns=['SMB'])
    odf['AREA'] = dfs.rgi_area_km2 * 1e6
    # Just take the calving rate and change its units
    # Original units: km3 a-1, to change to mm a-1 (units of specific MB)
    rho = cfg.PARAMS['ice_density']
    if 'calving_flux' in dfs:
        odf['CALVING'] = dfs['calving_flux'].fillna(0) * 1e9 * rho / odf['AREA']
    else:
        odf['CALVING'] = 0

    # We have to drop nans here, which occur when calving glaciers fail to run
    odf = odf.dropna()

    # Compare area with total RGI area
    rdf = 'rgi62_areas.csv'
    rdf = pd.read_csv(utils.get_demo_file(rdf), dtype={'O1Region': str})
    ref_area = rdf.loc[rdf['O1Region'] == rgi_reg].iloc[0]['AreaNoC2NoNominal']
    diff = (1 - odf['AREA'].sum() * 1e-6 / ref_area) * 100
    msg = 'Applying geodetic MB correction on RGI reg {}. Diff area: {:.2f}%'
    log.workflow(msg.format(rgi_reg, diff))

    # Total MB OGGM
    out_smb = np.average(odf['SMB'], weights=odf['AREA'])  # for logging
    out_cal = np.average(odf['CALVING'], weights=odf['AREA'])  # for logging
    smb_oggm = np.average(odf['SMB'] - odf['CALVING'], weights=odf['AREA'])

    # Total MB Reference
    df = 'table_hugonnet_regions_10yr_20yr_ar6period.csv'
    df = pd.read_csv(utils.get_demo_file(df))
    df = df.loc[df.period == '2006-01-01_2019-01-01'].set_index('reg')
    smb_ref = df.loc[int(rgi_reg), 'dmdtda']

    # Diff between the two
    residual = smb_ref - smb_oggm

    # Let's just shift
    log.workflow('Shifting regional MB bias by {}'.format(residual))
    log.workflow('Observations give {}'.format(smb_ref))
    log.workflow('OGGM SMB gives {}'.format(out_smb))
    log.workflow('OGGM frontal ablation gives {}'.format(out_cal))
    for gdir in gdirs:
        try:
            df = gdir.read_json('local_mustar')
            gdir.add_to_diagnostics('mb_bias_before_geodetic_corr', df['bias'])
            df['bias'] = df['bias'] - residual
            gdir.write_json(df, 'local_mustar')
        except FileNotFoundError:
            pass

## 1. Preprocess a subset of an RGI region
This example shows how to run the first steps of the OGGM preprocessing chain for a subset of the Alps - the Rofental catchment in the Austrian Alps (see [docs.oggm.org](http://docs.oggm.org/en/latest/run_examples/run_rgi_region.html)).

In [ ]:
# Python imports
import os

# Libs
import geopandas as gpd
import shapely.geometry as shpg

# Locals
from oggm import cfg, utils, workflow

In [ ]:
# Initialize OGGM and set up the default run parameters
cfg.initialize()
rgi_version = '62'
rgi_region = '11'  # Alps

# Local working directory (where OGGM will write its output)
wdir = utils.gettempdir('match_regional_mb')
utils.mkdir(wdir, reset=True)
cfg.PATHS['working_dir'] = wdir

In [ ]:
wdir

In [ ]:
# use intersects
path = utils.get_rgi_intersects_region_file(rgi_region, version=rgi_version)
cfg.set_intersects_db(path)

In [ ]:
# RGI file
path = utils.get_rgi_region_file(rgi_region, version=rgi_version)
rgidf = gpd.read_file(path)

# Get the Rofental Basin file
path = utils.get_demo_file('rofental_hydrosheds.shp')
basin = gpd.read_file(path)

In [ ]:
# Take all glaciers in the Rofental Basin
in_bas = [basin.geometry.contains(shpg.Point(x, y))[0] for
          (x, y) in zip(rgidf.CenLon, rgidf.CenLat)]
rgidf = rgidf.loc[in_bas]
# Store them for later
rgidf.to_file(os.path.join(cfg.PATHS['working_dir'], 'rgi_rofental.shp'))

In [ ]:
# Sort for more efficient parallel computing
rgidf = rgidf.sort_values('Area', ascending=False)
cfg.PARAMS['use_multiprocessing'] = True

print('Starting OGGM run')
print('Number of glaciers: {}'.format(len(rgidf)))

# Go - initialize glacier directories
gdirs_list = workflow.init_glacier_regions(rgidf)

In [ ]:
# select a number of glaciers
gdirs = gdirs_list.copy()  # selecting all

In [ ]:
from oggm.core import vascaling, climate
# use default climate (=CRU), for HISTALP uncomment the following lines
# cfg.PARAMS['baseline_climate'] = 'HISTALP'
# cfg.PARAMS['prcp_scaling_factor'] = 1.75
# cfg.PARAMS['temp_melt'] = -1.75

# run all GIS tasks
workflow.gis_prepro_tasks(gdirs)

In [ ]:
# execute climate tasks
workflow.execute_entity_task(climate.process_climate_data, gdirs);
workflow.execute_entity_task(vascaling.local_t_star, gdirs);

In [ ]:
# Compile output
utils.compile_glacier_statistics(gdirs)
utils.write_centerlines_to_shape(gdirs)

In [ ]:
# Imports
from os import path
import geopandas as gpd
import matplotlib.pyplot as plt
from oggm.utils import get_demo_file, gettempdir

# Local working directory (where OGGM wrote its output)
WORKING_DIR = cfg.PATHS['working_dir']

# Plot: the basin, the outlines and the centerlines
basin = gpd.read_file(get_demo_file('rofental_hydrosheds.shp'))
rgi = gpd.read_file(path.join(WORKING_DIR, 'rgi_rofental.shp'))
centerlines = gpd.read_file(path.join(WORKING_DIR, 'glacier_centerlines.shp'))

f, ax = plt.subplots()
basin.plot(ax=ax, color='k', alpha=0.2)
rgi.plot(ax=ax, color='C0')
centerlines.plot(ax=ax, color='C3')
plt.title('Rofental glaciers and centerlines')
plt.tight_layout()
plt.show()

## 2. Complete run for a list of glaciers
This example shows how to run the OGGM model for a list of selected glaciers.

Note that the default in OGGM is to use a previously calibrated list of $t^*$ for the run, which means that we don't have to calibrate the mass balance model ourselves (thankfully, otherwise you'd have to add all the calibration glaciers to your list too).

Note that in order to be correct, the automated calibration can only be used if the model parameters don't change between the calibration and the run. After testing, it appears that changing the `border` parameter won't affect the results much (as expected). So it's ok to change this parameters. Some other parameters (e.g. topo smoothing, dx, precipitation factor, alternative climate data, ...) will probaly need a re-calibration step (see [3. Run the mass balance calibration](http://docs.oggm.org/en/latest/run_examples/run_mb_calibration.html)).


In [ ]:
import importlib
importlib.reload(vascaling)
importlib.reload(workflow)

In [ ]:
# Random climate representative for the tstar climate, without bias
# In an ideal world this would imply that the glaciers remain stable,
# but it doesn't have to be so
nyears=200
workflow.execute_entity_task(vascaling.run_random_climate, gdirs,
                             nyears=nyears, bias=0, seed=12);

In [ ]:
import numpy as np
import pandas as pd
# create empty container
volume_m3 = pd.DataFrame(index=np.arange(0,nyears))
# iterate over all gdirs
for gdir in gdirs:
    path = gdir.get_filepath('model_diagnostics', filesuffix='vas')
    df = pd.read_csv(path, index_col=0)
    vol = df.volume_m3
    rgi_id = gdir.rgi_id
    volume_m3[rgi_id] = vol

In [ ]:
path

In [ ]:
volume_m3.sum(axis=1).plot()

## Test some stuff

In [ ]:
# Python imports
import os
import numpy as np
import pandas as pd
import geopandas as gpd
import shapely.geometry as shpg

# Locals
import oggm.cfg as cfg
from oggm import utils, workflow
from oggm.core import vascaling, climate

In [ ]:
# Initialize OGGM and set up the default run parameters
cfg.initialize()
rgi_version = '61'
rgi_region = '11'  # Alps

# Local working directory (where OGGM will write its output)

wdir = '/Users/oberrauch/master/working_sirectories/commitment_run_vas/'
utils.mkdir(wdir)
cfg.PATHS['working_dir'] = wdir

# We use intersects
path = utils.get_rgi_intersects_region_file(rgi_region, version=rgi_version)
cfg.set_intersects_db(path)

# RGI file
path = utils.get_rgi_region_file(rgi_region, version=rgi_version)
rgidf = gpd.read_file(path)

# Get the Rofental Basin file
path = utils.get_demo_file('rofental_hydrosheds.shp')
basin = gpd.read_file(path)

# Take all glaciers in the Rofental Basin
in_bas = [basin.geometry.contains(shpg.Point(x, y))[0] for
          (x, y) in zip(rgidf.CenLon, rgidf.CenLat)]
rgidf = rgidf.loc[in_bas]
# Store them for later
rgidf.to_file(os.path.join(wdir, 'rgi_rofental.shp'))

# Sort for more efficient parallel computing
rgidf = rgidf.sort_values('Area', ascending=False)
cfg.PARAMS['use_multiprocessing'] = True

# Go - initialize glacier directories
gdirs = workflow.init_glacier_regions(rgidf)

In [ ]:
# select Hintereisferener
gdir = gdirs[0]
cfg.PARAMS['baseline_climate'] = 'HISTALP'
cfg.PARAMS['prcp_scaling_factor'] = 1.75
cfg.PARAMS['temp_melt'] = -1.75
vascaling.run_random_vas_climate(gdir, y0=2000)

In [ ]:
path = gdir.get_filepath('model_diagnostics', filesuffix='vas')
df = pd.read_csv(path, index_col=0)
df.head()

In [ ]:
df.volume_m3.plot()

### Access `model_diagnostice` from already run gdirs

In [ ]:
# Python imports
import os
import numpy as np
import pandas as pd
import geopandas as gpd
import shapely.geometry as shpg

# Locals
import oggm.cfg as cfg
from oggm import utils, workflow
from oggm.core import vascaling, climate

In [ ]:
# Initialize OGGM and set up the default run parameters
cfg.initialize()
rgi_version = '61'
rgi_region = '11'  # Alps

# Local working directory (where OGGM will write its output)

wdir = '/Users/oberrauch/master/working_sirectories/commitment_run_vas/'
utils.mkdir(wdir)
cfg.PATHS['working_dir'] = wdir

# We use intersects
path = utils.get_rgi_intersects_region_file(rgi_region, version=rgi_version)
cfg.set_intersects_db(path)

# RGI file
path = utils.get_rgi_region_file(rgi_region, version=rgi_version)
rgidf = gpd.read_file(path)

# Get the Rofental Basin file
path = utils.get_demo_file('rofental_hydrosheds.shp')
basin = gpd.read_file(path)

# Take all glaciers in the Rofental Basin
in_bas = [basin.geometry.contains(shpg.Point(x, y))[0] for
          (x, y) in zip(rgidf.CenLon, rgidf.CenLat)]
rgidf = rgidf.loc[in_bas]
# Store them for later
rgidf.to_file(os.path.join(wdir, 'rgi_rofental.shp'))

# Sort for more efficient parallel computing
rgidf = rgidf.sort_values('Area', ascending=False)
cfg.PARAMS['use_multiprocessing'] = True

# Go - initialize glacier directories
gdirs = workflow.init_glacier_regions(rgidf)

In [ ]:
nyears = 300
# create empty container
volume_m3 = pd.DataFrame(index=np.arange(0,nyears))
# iterate over all gdirs
for gdir in gdirs:
    path = gdir.get_filepath('model_diagnostics', filesuffix='vas')
    df = pd.read_csv(path, index_col=0)
    vol = df.volume_m3
    rgi_id = gdir.rgi_id
    volume_m3[rgi_id] = vol

In [ ]:
volume_m3.sum(axis=1).plot()

In [ ]:
# Imports
import os
import xarray as xr
import matplotlib.pyplot as plt
from oggm.utils import get_demo_file, gettempdir

# Local working directory (where OGGM wrote its output)
wdir = '/Users/oberrauch/master/working_sirectories/commitment_run_oggm/'
# Read the files using xarray
ds = xr.open_dataset(os.path.join(wdir, 'run_output_commitment.nc'))

In [ ]:
# Volume
(ds.volume.sum(dim='rgi_id')).plot(label='OGGM')
volume_m3.sum(axis=1).plot(label='VAS')
plt.legend()
plt.title('Overall glacier volume - Rofental')
plt.xlabel('')
plt.ylabel('Volume [m$^3$]')

In [ ]:
# Local working directory (where OGGM will write its output)
wdir = '/Users/oberrauch/master/working_sirectories/commitment_run_oggm/'
utils.mkdir(wdir, reset=False)
cfg.PATHS['working_dir'] = wdir
gdirs = workflow.init_glacier_regions()

In [ ]:
from oggm.core import flowline
flowline.init_present_time_glacier(gdirs[1])

In [ ]:
cfg.PARAMS['baseline_climate'] = 'HISTALP'
cfg.PARAMS['prcp_scaling_factor'] = 1.75
cfg.PARAMS['temp_melt'] = -1.75
flowline.run_random_climate(gdirs[1], nyears=300, y0=1999, seed=2,
                            output_filesuffix='_commitment')

In [ ]:
path = gdirs[1].get_filepath('model_diagnostics', filesuffix='_commitment')
path

In [ ]:
nyears = 300
# create empty container
volume_m3 = pd.DataFrame(index=np.arange(0,nyears))
# iterate over all gdirs
for gdir in gdirs:

    df = pd.read_csv(path, index_col=0)
    vol = df.volume_m3
    rgi_id = gdir.rgi_id
    volume_m3[rgi_id] = vol

In [ ]:
path

In [ ]:
import xarray as xa
ds = xa.open_dataset(path)
ds.volume_m3.plot()

### Quick test of RandomClimate

In [ ]:
# quick-test of the RandomVASMassBalanceModel
import importlib
importlib.reload(vascaling)

In [ ]:
import numpy as np

In [ ]:
# select Hintereisferner gdir
gdir = gdirs[int(np.where([gdir.name == 'Hintereisferner' for gdir in gdirs])[0])]
gdir

In [ ]:
rand_mbmod = vascaling.RandomVASMassBalance(gdir=gdir, seed=12, y0=1999)

In [ ]:
min_hgt, max_hgt = vascaling.get_min_max_elevation(gdir)
year = 2043
r_yr = rand_mbmod.get_state_yr(year)
mb = rand_mbmod.get_specific_mb(min_hgt, max_hgt, year)
print('Year {} correpsonds to mb year {}, resulting in {} mm w.e. yr-1'.format(year, r_yr, mb))

In [ ]:
years = np.arange(2018, 2200)
mb = np.empty(years.size)
r_yrs = np.empty(years.size)
for i, year in enumerate(years):
    mb[i] = rand_mbmod.get_specific_mb(min_hgt, max_hgt, year)
    r_yrs[i] = rand_mbmod.get_state_yr(year)

import pandas as pd
df = pd.DataFrame({'mb':mb, 'r_yr':r_yrs}, index=years)

In [ ]:
df.mb.plot()

In [ ]:
local_tstar = gdir.read_json('vascaling_mustar')
t_star = local_tstar['t_star']

In [ ]:
import matplotlib.pyplot as plt
df.r_yr.plot()
plt.axhline(t_star-15)
plt.axhline(t_star+15)

In [ ]:
ls /Users/oberrauch/master/working_sirectories/commitment_run_vas/per_glacier/RGI60-11/RGI60-11.00/

### Store to xarray

In [ ]:
path = '/Users/oberrauch/master/working_sirectories/commitment_run_oggm/'+\
    'per_glacier/RGI60-11/RGI60-11.00/RGI60-11.00731/model_run_commitment.nc'
import xarray as xr
xr.open_dataarray(path)

In [ ]:
path = '/Users/oberrauch/master/working_sirectories/commitment_run_vas/'+\
    'per_glacier/RGI60-11/RGI60-11.00/RGI60-11.00739/model_diagnosticsvas.nc'
df = pd.read_csv(path, index_col=0)
df.head()

In [ ]:
from time import gmtime, strftime
import xarray as xr

In [ ]:
diag_ds = xr.Dataset()

# Global attributes
diag_ds.attrs['description'] = 'OGGM model output'
# diag_ds.attrs['oggm_version'] = __version__
diag_ds.attrs['calendar'] = '365-day no leap'
diag_ds.attrs['creation_date'] = strftime("%Y-%m-%d %H:%M:%S",
                                          gmtime())


diag_ds

In [ ]:
# Coordinates
diag_ds.coords['time'] = ('time', monthly_time)
diag_ds.coords['hydro_year'] = ('time', yrs)
diag_ds.coords['hydro_month'] = ('time', months)
diag_ds.coords['calendar_year'] = ('time', cyrs)
diag_ds.coords['calendar_month'] = ('time', cmonths)

In [ ]:
utils.monthly_timeseries(20, 10)

In [ ]:
# time
        yearly_time = np.arange(np.floor(self.yr), np.floor(y1)+1)

        if store_monthly_step:
            monthly_time = utils.monthly_timeseries(self.yr, y1)
        else:
            monthly_time = np.arange(np.floor(self.yr), np.floor(y1)+1)
        yrs, months = utils.floatyear_to_date(monthly_time)
        cyrs, cmonths = utils.hydrodate_to_calendardate(yrs, months)